# Import

In [1]:
import numpy as np

# Define

In [34]:
class VariableMatrix():
    def __init__(self, size, mean = 0, standar_d = 1, work_components = 1, is_independent_learning = False):
        self.size = size
        self.values = np.random.nromal(mean, standar_d, size)
        self.diff = np.zeros(size)
        self.drop = np.ones(size, dtype = bool)
        
        if work_components == 1:
            self.work = np.ones(size, dtype = bool)
        else:
            self.work = (np.random.random(size) < work_components)
        
        if is_independent_learning:
            self.eta_m = np.zeros(size)
        else:
            self.eta_m = None

    def move(self, eta = None):
        move_values = (-1) * self.diff * self.works * self.drop
        if eta == None:
            self.values += self.eta_m * move_values
        else:
            self.values += eta * move_values
            
        
                    
class Sigmoid():
    def trans(self, x):
        out_put = np.zeros([len(x)])
        for i in range(len(x)):
            out_put[i] = 1 / (1 + np.exp(-x[i]))
        
        return out_put
    
    def diff(self, x):
        out_put = np.zeros([len(x), len(x)])
        for i in range(len(x)):
            out_put[i][i] = 1 / (1 + np.exp(x[i]) * (1 + np.exp(-x[i])))
            
        return out_put
    
class Identity():
    def trans(self, x):
        return x
    
    def diff(self, x):
        return np.identity(len(x))

class Layer():
    def __init__(self, layers_from, nodes_n, active_function):
        self.lf = layers_from # list :layers connected before
        self.lt = [] # layers connected after
        self.nodes_n = nodes_n
        self.weights = {}
        self.bias = VariableMatrix([nodes_n, 1])
        self.nodes_before = np.zeros([nodes_n, 1])
        self.nodes_after = np.zeros([nodes_n, 1])
        self.nodes_diff = np.zeros([nodes_n, 1])
        self.af = active_function
    
    def forward(self):
        self.nodes_after = self.af.trans(self.nodes_before)
    
    def jacobi(self):
        self.nodes_diff = self.af.diff(self.nodes_before)
    
class DogikoNN():
    def __init__(self):
        self.training_data = None
        self.pp_linear = {
            "multi" : None,
            "trans" : None
        } # pre-processing linear translation
        self.flow = []
        self.layers = {}
    
    def set_data(self, input_data, input_answer, input_weights = None):
        # type input_data : numpy array
        # type input_answer : numpy array
        self.t_datas = input_data
        self.t_answers = input_answer
        self.t_weights = input_weights
        
    def define_normalized(self, normalized_algorithm):
        self.normal_alg = normalized_algorithm
    
    def layers_clear(self):
        del self.layers
        self.layers = {}
        
    def insert_layer(self, name, layer, is_flow_add = True):
        self.layers[name] = layer
        if is_flow_add:
            self.flow.append(name)
        
    def build(self):
        if self.normal_alg == "normal":
            temp_trans_datas = self.t_datas.T
            temp_square_mean_array = np.zeros([len(temp_trans_datas)])
            self.pp_linear["trans"] = np.zeros([len(temp_trans_datas)])
            for j in range(len(temp_trans_datas)):
                feature = temp_trans_datas[j]
                self.pp_linear["trans"][j] = feature.mean()
                temp_square_mean_array[j] = (feature**2).mean()
            
            temp_sd = (temp_square_mean_array - self.pp_linear["trans"]**2)**0.5
            temp_sd[temp_sd == 0] = 1
            self.pp_linear["multi"] = 1 / temp_sd
            
        else:
            print ("unacceptable normalized_algorithm")
            return ("unacceptable normalized_algorithm")
        
        self.n_t_datas = np.zeros(self.t_datas.shape)
        for i in range(len(self.t_datas)):
            self.n_t_datas[i] = self.pp_linear["multi"] * (self.t_datas[i] - self.pp_linear["trans"])
        
        if len(self.flow) != len(set(self.flow)):
            print ("each element in .flow should be unique (layer name)")
            return ("each element in .flow should be unique (layer name)")
        
        for l in range(len(self.flow)):
            ln = self.flow[l]
            self.layers[ln].lt = []
            for ln2 in self.layers[ln].lf:
                if self.flow.index(ln2) > l:
                    print ("layer " + ln2 + " shoud work before layer " + ln)
                    return ("layer " + ln2 + " shoud work before layer " + ln)
                    
                self.layers[ln2].lt.append(ln)
                self.layers[ln].weights[ln2] = VariableMatrix([self.layers[ln].nodes_n, self.layers[ln2].nodes_n])
        
        

In [175]:
try_NN = DogikoNN()

try_data = np.random.rand(*[3, 2]) * 2 - 1
try_answer = np.random.randint(2, size = [3, 1])
try_NN.set_data(try_data, try_answer)

try_NN.define_normalized("normal")

try_NN.insert_layer(
    'A',
    Layer(
        layers_from = [],
        nodes_n = 2,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'B',
    Layer(
        layers_from = ['A'],
        nodes_n = 3,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'C',
    Layer(
        layers_from = ['A'],
        nodes_n = 4,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'D',
    Layer(
        layers_from = ['B', 'C'],
        nodes_n = 5,
        active_function = Sigmoid()
    )
)

In [179]:
try_NN.build()

In [180]:
try_NN.layers['A'].lt

['B', 'C']